In [1]:
# Import storiks.plot.
from storiks import run

# Import some additional packages and set the absolute path of the current directory.
import glob
import os
curdir = os.path.abspath('.')

In [2]:
# List files in the current directory.
# /output   - experiment output directory (--output_dir informed in storiksctl start)
# /workdata - data directory (--data_dir informed in storiksctl start)
!pwd
!ls

/output/examples
base.flags
base_pressure_iodepth.flags
create-ycsb.flags
exp_01-ycsb_workloadb.flags
exp_01-ycsb_workloadb.label
exp_02-ycsb_workloadb-pressure_iodepth_n1bs4rr100wr000.flags
exp_02-ycsb_workloadb-pressure_iodepth_n1bs4rr100wr000.label
exp_03-iodepth_bs4rr100wr000.flags
plot.ipynb
run.ipynb


In [3]:
# List the scheduled tasks
run.list()

num.  status      start                end                  command
----------------------------------------------------------------------------------------



In [4]:
# Create one or more files used by access_time3 (concurrent workloads)
!mkdir /workdata/at3
run.schedule(
    'access_time3 --create_file --duration=5 --filename=/workdata/at3/0 --filesize=10240',
    output=f'{curdir}/create-at3.txt', append=True)

command 0 scheduled



In [5]:
# Create a RocksDB database using YCSB
!mkdir /workdata/rocksdb_ycsb_0
run.schedule(
    f'storiks --flagfile={curdir}/create-ycsb.flags',
    output=f'{curdir}/create-ycsb.out')

command 1 scheduled



In [6]:
# Schedule all storiks experiments in the current directory (exp*.flags)
for f in glob.glob('exp*.flags'):
    print(f'Scheduling {f}:')
    fprefix = f.replace('.flags', '')
    run.schedule(f'storiks --flagfile={curdir}/{f}', output=f'{curdir}/{fprefix}.out')  # run the experiment
    run.schedule(f"bash -c 'xz -9 {curdir}/*.out'")  # compress the output file
    run.schedule("sleep 15")  # wait 15 seconds before the next experiment

Scheduling exp_01-ycsb_workloadb.flags:
command 2 scheduled

command 3 scheduled

command 4 scheduled

Scheduling exp_02-ycsb_workloadb-pressure_iodepth_n1bs4rr100wr000.flags:
command 5 scheduled

command 6 scheduled

command 7 scheduled

Scheduling exp_03-iodepth_bs4rr100wr000.flags:
command 8 scheduled

command 9 scheduled

command 10 scheduled



In [7]:
# Check the scheduled tasks again
run.list()

num.  status      start                end                  command
----------------------------------------------------------------------------------------
   0  running     2021-09-17 18:52:56                       access_time3 --create_file --duration=5 --file ...
   1  scheduled                                             storiks --flagfile=/output/examples/create-ycs ...
   2  scheduled                                             storiks --flagfile=/output/examples/exp_01-ycs ...
   3  scheduled                                             bash -c 'xz -9 *.out'
   4  scheduled                                             sleep 15
   5  scheduled                                             storiks --flagfile=/output/examples/exp_02-ycs ...
   6  scheduled                                             bash -c 'xz -9 *.out'
   7  scheduled                                             sleep 15
   8  scheduled                                             storiks --flagfile=/output/examples/e

In [8]:
# Inspect the status of a particular task.
run.status(1)

ScheduledCommand:
	    number = 1
	   command = storiks --flagfile=/output/examples/create-ycsb.flags
	    status = running
	     start = 2021-09-17 18:53:07
	       end = 
	  exitcode = 
	  fail_msg = 



In [9]:
# Or even cancel any unfinished task.
run.cancel(10)

command number 10 scheduled to cancel

